In [1]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
from collections import Counter

In [2]:
try:
  cnx = mysql.connector.connect(user='admin', password='qazwsx&*%^',
                              host='shadowfax1.cedjgswziywb.ap-south-1.rds.amazonaws.com',
                              database='shadowfax')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

In [3]:
db_cursor = cnx.cursor()
# db_cursor.execute('SELECT * FROM products')
df = pd.read_sql('SELECT * FROM shadowfax.order_history INNER JOIN shadowfax.products ON shadowfax.products.inc_id=shadowfax.order_history.item_id;', con=cnx)
# df['id'] = df['inc_id']
user = '2'
# df.set_index('order_history.inc_id', inplace=True)
df = df.loc[df['user_id'] == user]
df = df[['order_date','product_name','user_id']]
df

,order_date,product_name,user_id
0,1625509800,Glucon-D,2
1,1628188200,Glucon-D,2
2,1628533800\t,Curd,2
3,1629138600\t,Curd,2
4,1629743400,Curd,2
5,1630348200,Curd,2
6,1630434600,Red lays,2
7,1630434600,Blue Lays,2
8,1630434600,Pepsi,2
9,1630002600\t,Maggi,2


In [4]:
import datetime
def change2month(timestamp):
    ts = datetime.datetime.fromtimestamp(int(timestamp))
    m = int(ts.strftime('%m'))
    return m

def change2wk(timestamp):
    ts = datetime.datetime.fromtimestamp(int(timestamp))
    w = int(ts.strftime('%V'))
    return w

In [5]:
df['month'] = df.apply(lambda row: change2month(row.order_date), axis=1)
df['week'] = df.apply(lambda row: change2wk(row.order_date), axis=1)
df

,order_date,product_name,user_id,month,week
0,1625509800,Glucon-D,2,7,27
1,1628188200,Glucon-D,2,8,31
2,1628533800\t,Curd,2,8,32
3,1629138600\t,Curd,2,8,33
4,1629743400,Curd,2,8,34
5,1630348200,Curd,2,8,35
6,1630434600,Red lays,2,9,35
7,1630434600,Blue Lays,2,9,35
8,1630434600,Pepsi,2,9,35
9,1630002600\t,Maggi,2,8,34


In [6]:
products = df['product_name'].unique().tolist()
products

['Glucon-D',
 'Curd',
 'Red lays',
 'Blue Lays',
 'Pepsi',
 'Maggi',
 'Dairy Milk',
 'Pazzta']

In [7]:
for prod in products:
    df_indiv = df.loc[df['product_name'] == prod]['order_date']

In [8]:
df_indiv = df.loc[df['product_name'] == 'Curd']['week'].tolist()
df_indiv

[32, 33, 34, 35]

In [9]:
df_month = df.loc[df['product_name'] == 'Glucon-D']['month'].tolist()
df_month

[7, 8]

In [10]:
ts = datetime.datetime.today()
w = int(ts.strftime('%V'))
m = int(ts.strftime('%m'))
m,w

(9, 36)

In [11]:
def week_recurring(wk_list):
    if w - wk_list[len(wk_list)-1] in [1,2] and w - wk_list[len(wk_list)-2] in [2,3]:
        print("Yes")
def month_recurring(month_list):
    if m - month_list[len(month_list)-1] == 1 and m - month_list[len(month_list)-2] == 2:
        print("Yes")
week_recurring(df_indiv)
month_recurring(df_month)

Yes
Yes


In [15]:
def week_recurring(w, wk_list):
    if w - wk_list[len(wk_list)-1] in [1,2] and w - wk_list[len(wk_list)-2] in [2,3]:
        return True
def month_recurring(m, month_list):
    if m - month_list[len(month_list)-1] == 1 and m - month_list[len(month_list)-2] == 2:
        return True

def get_time_arrays(df):
    products = df['product_name'].unique().tolist()
    ts = datetime.datetime.today()
    w = int(ts.strftime('%V'))
    m = int(ts.strftime('%m'))
    weekly_recurring = []
    monthly_recurring = []
    for prod in products:
        df_week = df.loc[df['product_name'] == prod]['week'].tolist()
        df_month = df.loc[df['product_name'] == prod]['month'].tolist()
        if week_recurring(w, df_week):
            weekly_recurring.append(prod)
        if month_recurring(m, df_month):
            monthly_recurring.append(prod)
            
    return weekly_recurring, monthly_recurring

In [17]:
weeks, months = get_time_arrays(df)
weeks, months

(['Curd'], ['Glucon-D'])